In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/students-performance-in-exams/StudentsPerformance.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
# import pandas_profiling as pp
# pp.ProfileReport(data)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
sns.set_style('dark')

mask = np.zeros_like(data.corr())
mask[np.triu_indices_from(mask)] = True
sns.heatmap(data.corr(), square=True, cmap=sns.color_palette("rocket"), mask=mask, annot=True)

Since none of the values are missing, might as well starting doing the 
# Machine Learning Section

First though, gotta figure out which features need to be transformed how. 
* The first 5 columns are categorical, the last 3 are continuous (but since we are predicting math score, we can just delete reading and writing). 
* Just do get_dummies for the categorical ones or OneHotEncoder if you are feeling spicy (probably better if you plan on deploying this model online with a pipeline).
* So, we basically just have to figure out what the average is for the math score and then set the outcome to 0 or 1 depending on if the student is above average.
* Also, there are no missing values, which is great as always.

In [ ]:
train_vars = ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course']
train = data[train_vars]
train.head()

In [ ]:
data['math score'].mean()

This is a bit of feature engineering below, just trying to turn this regression problem into a classification problem so that it fits the format. 

In [ ]:
data['Outcome'] = 0
data.loc[data['math score'] >= 66.089, 'Outcome'] = 1
data.head()

In [ ]:
target = data.Outcome
target

Now let's just get the dummy variables for the testing data

In [ ]:
train = pd.get_dummies(train)
train.head()

# Train-test split time

In [ ]:
train.shape

In [ ]:
target.shape

In [ ]:
from sklearn.model_selection import train_test_split

train, test_train, target, test_target = train_test_split(train, target, test_size=0.2, random_state=0)
print(train.info())
print(test_train.info())

In [ ]:
# train, val_train, target, val_target = train_test_split(train, target, test_size=0.2)
# train.info()
# val_train.info()

# XGBoost Time

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
estimator = XGBClassifier(
    objective = 'binary:logistic',
    seed = 1337
)
estimator

In [ ]:
params = {
    'max_depth': range(1, 10, 1),
    'n_estimators': range(50, 700, 40),
    'learning_rate': [x/100 for x in range(1, 15, 2)]
}
params

Best parameters after search:
max_depth: 1, n_estimators: 490, learning_rate: 0.05

In [ ]:
# gs = GridSearchCV(estimator=estimator, param_grid=params, scoring='roc_auc', n_jobs=-1, cv=5, verbose=True)

In [ ]:
# gs.fit(train, target)

In [ ]:
# model = gs.best_estimator_
# model

In [ ]:
model = XGBClassifier(objective='binary:logistic', seed=1337, max_depth=1, n_estimators=490, learning_rate=0.05)
model.fit(train, target)

In [ ]:
from sklearn.metrics import accuracy_score
prediction = model.predict(test_train)
prediction = [round(p) for p in prediction]

In [ ]:
accuracy = accuracy_score(prediction, test_target)
print(accuracy*100)

Accuracy is about 58-68%, which shows that the XGBoost algorithm can only get about a passing grade on predicting passing grades. 

# Logistic Regression Time

In [ ]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()
params = {
    'C': [0.001, 0.01, 0.02, 0.03, 0.05, 0.07, 0.1, 0.2],
    'penalty': ['l1', 'l2'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
}
params

In [ ]:
gs = GridSearchCV(estimator=logReg, param_grid=params, n_jobs=-1, cv=5, verbose=True)

In [ ]:
gs.fit(train, target)

In [ ]:
model = gs.best_estimator_
model

Best option is: C=0.03, solver='newton-cg'

In [ ]:
model = LogisticRegression(C=0.03, solver='newton-cg')
model.fit(train, target)

In [ ]:
prediction = model.predict(test_train)
accuracy = accuracy_score(prediction, test_target)
print(accuracy*100)

Accuracy does not go up